## Load the model

In [2]:
import os
import gensim.models as g
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn.decomposition import PCA as sklearnPCA
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import pandas as pd

In [5]:
model="../../../doc2vec.bin"

#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

#load model
m = g.Doc2Vec.load(model)

In [18]:
r = open('venues_animals.json','r',encoding='utf-8')
w = open('animalsResult.json','w',encoding='utf-8')
X = []

count = 0
    
for line in r:
    a = line.split(' ')
    vector = line.split(' ')
#     if vector[1] != "\n":
    vector = vector[0:len(vector)-1]

    if len(vector) > 0:
        inferred_vector = m.infer_vector(vector, alpha=start_alpha, steps=infer_epoch)
        X.append(inferred_vector)
    
        for element in inferred_vector:
            w.write(str(element))
            w.write(' ')
        w.write('\n')
    else :
        print(a)
        count += 1
print(count)
r.close()
w.close()
X = np.array(X)
np.save('Xa.npy', X) 

0


In [3]:
X = np.load('Xa.npy')
print(len(X))

3530


## Setez labels ca sa pot distinge cele doua clase

In [4]:
cl = []
for i in range (0,3520):
    cl.append(1)
for i in range(0,10):
    cl.append(2)

In [5]:
print(len(cl))

3530


In [6]:
with open("cl.tsv", "w") as f:
    for elem in cl:
        print(str(elem), file=f)

In [7]:
PATH = os.getcwd()

LOG_DIR = PATH + '.'

metadata = os.path.join(LOG_DIR, 'cl.tsv')

pca = sklearnPCA(n_components=50) #2-dimensional PCA
pca_df = pd.DataFrame(pca.fit_transform(X))
df_pca = pca_df.values

tf_data = tf.Variable(df_pca)

with tf.Session() as sess:
    saver = tf.train.Saver([tf_data])
    sess.run(tf.global_variables_initializer())
    saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
    config = projector.ProjectorConfig()
    
    embedding = config.embeddings.add()
    embedding.tensor_name = tf_data.name
    
    embedding.metadata_path = metadata
    
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

Instructions for updating:
Colocations handled automatically by placer.


 ## Check DBSCAN

In [28]:
eps = [1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,
      2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,
      3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,
      4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,
      5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,
      6.1,6.2,6.3,6.4,6.5,6.6,6.7,6.8,6.9,
      7.1,7.2,7.3,7.4,7.5,7.6,7.7,7.8,7.9,
      8.1,8.2,8.3,8.4,8.5,8.6,8.7,8.8,8.9,
      9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,
      10.1,10.2,10.3,10.4,10.5,10.6,10.7,10.8,10.9]
i = 0
while i < 100:
    min_s = 1
    while min_s < 10:
        clustering = DBSCAN(eps=eps[i], min_samples=min_s).fit(X)
        l = []
        cl = clustering.labels_
        min_ones = 0
        for element in cl:
            if element == -1:
                min_ones = min_ones + 1
            if element not in l:
                l.append(element)
        print('eps=' + str(eps[i]) + ' min_s=' + str(min_s) + ' nr_clusters=' + str(len(l)) + ' nr_-1=' + str(min_ones))
        min_s = min_s + 1
    i = i + 1

eps=1.1 min_s=1 nr_clusters=2917 nr_-1=0
eps=1.1 min_s=2 nr_clusters=187 nr_-1=2731
eps=1.1 min_s=3 nr_clusters=66 nr_-1=2973
eps=1.1 min_s=4 nr_clusters=29 nr_-1=3128
eps=1.1 min_s=5 nr_clusters=22 nr_-1=3194
eps=1.1 min_s=6 nr_clusters=15 nr_-1=3266
eps=1.1 min_s=7 nr_clusters=12 nr_-1=3297
eps=1.1 min_s=8 nr_clusters=8 nr_-1=3340
eps=1.1 min_s=9 nr_clusters=6 nr_-1=3368
eps=1.2 min_s=1 nr_clusters=2615 nr_-1=0
eps=1.2 min_s=2 nr_clusters=194 nr_-1=2422
eps=1.2 min_s=3 nr_clusters=70 nr_-1=2670
eps=1.2 min_s=4 nr_clusters=35 nr_-1=2834
eps=1.2 min_s=5 nr_clusters=20 nr_-1=2949
eps=1.2 min_s=6 nr_clusters=9 nr_-1=3037
eps=1.2 min_s=7 nr_clusters=9 nr_-1=3082
eps=1.2 min_s=8 nr_clusters=7 nr_-1=3146
eps=1.2 min_s=9 nr_clusters=6 nr_-1=3200
eps=1.3 min_s=1 nr_clusters=2279 nr_-1=0
eps=1.3 min_s=2 nr_clusters=159 nr_-1=2121
eps=1.3 min_s=3 nr_clusters=56 nr_-1=2327
eps=1.3 min_s=4 nr_clusters=35 nr_-1=2477
eps=1.3 min_s=5 nr_clusters=20 nr_-1=2597
eps=1.3 min_s=6 nr_clusters=12 nr_-1=271

KeyboardInterrupt: 

In [36]:
min_s = 2
eps = 2.1
clustering = DBSCAN(eps=eps, min_samples=min_s).fit(X)
l = []
clust = clustering.labels_
for element in clust:
    if element not in l:
        l.append(element)
print(' nr_clusters=' + str(len(l)) )
print( clust[len(clust)-11:] )

 nr_clusters=15
[ 0  0  0  0  0  0  0 -1  0  0  0]


In [42]:
print (l)
m = 0
for e in clust:
    if e == 7:
        print(m)
    m += 1
print(clust[0:100])
print(clust[2098])

[0, -1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
2088
2098
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0 -1]
7


In [35]:
with open("clust.tsv", "w") as f:
    for elem in clust:
        print(str(elem), file=f)
print (clust[8])

0


In [ ]:
with open("projector-tensorboard/log-2.9-1/cl.tsv", "w") as f:
    for elem in cl:
        print(str(elem), file=f)

In [25]:
with open("output2.tsv", "w") as f:
    i = 0
    for x in X:
        strline = ""
        for elem in x:
            strline = strline + str(elem) + "\t"
        strline = strline
        print(strline, file=f)
        i = i+1
    print(i)
#print(X[0])

3183


In [22]:
clustering = DBSCAN(eps=2.7, min_samples=2).fit(X)
cl = clustering.labels_
with open("projector-tensorboard/log-1/output2cl.tsv", "w") as f:
    for elem in cl:
        print(str(elem), file=f)

# For Tensorboard

In [11]:
PATH = os.getcwd()

LOG_DIR = PATH + '/projector-tensorboard/log-1/'

metadata = os.path.join(LOG_DIR, 'output2cl.tsv')

pca = sklearnPCA(n_components=50) #2-dimensional PCA
pca_df = pd.DataFrame(pca.fit_transform(X))
df_pca = pca_df.values

tf_data = tf.Variable(df_pca)

with tf.Session() as sess:
    saver = tf.train.Saver([tf_data])
    sess.run(tf.global_variables_initializer())
    saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
    config = projector.ProjectorConfig()
    
    embedding = config.embeddings.add()
    embedding.tensor_name = tf_data.name
    
    embedding.metadata_path = metadata
    
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

Instructions for updating:
Colocations handled automatically by placer.


In [33]:
pca = sklearnPCA(n_components=50) #2-dimensional PCA
transformed = pd.DataFrame(pca.fit_transform(X))


In [34]:
transformed = np.array(transformed)
print(transformed[0])

[-0.4514514  -0.8970818  -0.5427182   0.280583    0.32324618  0.36332858
  0.89380044  0.17968972 -0.3282872   0.16508476  0.34474862  0.15171908
 -0.07575375 -0.58325964  0.21215254 -0.8442374   0.22676082 -0.28760123
  0.12260294 -0.01905518 -0.12728168  0.04444432 -0.19466023 -0.21238996
  0.01315045 -0.28498283  0.00901939 -0.1185576   0.10274967  0.24984224
  0.07525621 -0.01458099 -0.21885602 -0.11952911 -0.18631175  0.06916519
 -0.3460794  -0.08657125  0.03402223 -0.00292605 -0.16791156  0.09601615
 -0.14634167 -0.03025275  0.08442162 -0.02770769  0.07381632 -0.06703213
 -0.08358905 -0.206986  ]


In [35]:
with open("outputPCA50.tsv", "w") as f:
    for x in transformed:
        strline = ""
        for elem in x:
            strline = strline + str(elem) + "\t"
        strline = strline + "\n"
        print(strline, file=f)

In [12]:
from pyclustering.cluster.dbscan import dbscan
from pyclustering.cluster import cluster_visualizer_multidim
from pyclustering.utils import read_sample

# Sample for cluster analysis.
sample = X

eps = [1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,
      2.0,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,
      3.0,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,
      4.0,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,
      5.0,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,
      6.0,6.1,6.2,6.3,6.4,6.5,6.6,6.7,6.8,6.9,
      7.0,7.1,7.2,7.3,7.4,7.5,7.6,7.7,7.8,7.9,
      8.0,8.1,8.2,8.3,8.4,8.5,8.6,8.7,8.8,8.9,
      9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,
      10.0,10.1,10.2,10.3,10.4,10.5,10.6,10.7,10.8,10.9]
i = 0
while i < 100:
    min_s = 1
    while min_s < 10:

        dbscan_instance = dbscan(sample, eps[i], min_s)
        dbscan_instance.process()
        clusters = dbscan_instance.get_clusters()
        noise = dbscan_instance.get_noise()
        print(str(eps[i]) + ' ' + str(min_s) + ' clusters=' + str(len(clusters)) + ' noise=' + str(len(noise)))
        min_s += 1
    
    i += 1


1.0 1 clusters=50 noise=3071
1.0 2 clusters=5 noise=3161
1.0 3 clusters=3 noise=3167
1.0 4 clusters=2 noise=3171
1.0 5 clusters=2 noise=3171
1.0 6 clusters=0 noise=3183
1.0 7 clusters=0 noise=3183
1.0 8 clusters=0 noise=3183
1.0 9 clusters=0 noise=3183
1.1 1 clusters=56 noise=3059
1.1 2 clusters=5 noise=3161
1.1 3 clusters=3 noise=3167
1.1 4 clusters=2 noise=3171
1.1 5 clusters=2 noise=3171
1.1 6 clusters=0 noise=3183
1.1 7 clusters=0 noise=3183
1.1 8 clusters=0 noise=3183
1.1 9 clusters=0 noise=3183
1.2 1 clusters=67 noise=3030
1.2 2 clusters=10 noise=3144
1.2 3 clusters=4 noise=3163
1.2 4 clusters=2 noise=3171
1.2 5 clusters=2 noise=3171
1.2 6 clusters=0 noise=3183
1.2 7 clusters=0 noise=3183
1.2 8 clusters=0 noise=3183
1.2 9 clusters=0 noise=3183
1.3 1 clusters=89 noise=2977
1.3 2 clusters=19 noise=3117
1.3 3 clusters=4 noise=3163
1.3 4 clusters=2 noise=3171
1.3 5 clusters=2 noise=3171
1.3 6 clusters=0 noise=3183
1.3 7 clusters=0 noise=3183
1.3 8 clusters=0 noise=3183
1.3 9 clusters

4.2 7 clusters=1 noise=81
4.2 8 clusters=1 noise=82
4.2 9 clusters=1 noise=83
4.3 1 clusters=3 noise=55
4.3 2 clusters=1 noise=59
4.3 3 clusters=1 noise=61
4.3 4 clusters=1 noise=63
4.3 5 clusters=1 noise=63
4.3 6 clusters=1 noise=64
4.3 7 clusters=1 noise=65
4.3 8 clusters=1 noise=65
4.3 9 clusters=1 noise=65
4.4 1 clusters=2 noise=41
4.4 2 clusters=1 noise=43
4.4 3 clusters=1 noise=45
4.4 4 clusters=1 noise=46
4.4 5 clusters=1 noise=47
4.4 6 clusters=1 noise=47
4.4 7 clusters=1 noise=47
4.4 8 clusters=1 noise=48
4.4 9 clusters=1 noise=48
4.5 1 clusters=2 noise=33
4.5 2 clusters=1 noise=35
4.5 3 clusters=1 noise=37
4.5 4 clusters=1 noise=38
4.5 5 clusters=1 noise=38
4.5 6 clusters=1 noise=38
4.5 7 clusters=1 noise=39
4.5 8 clusters=1 noise=39
4.5 9 clusters=1 noise=39
4.6 1 clusters=2 noise=28
4.6 2 clusters=1 noise=30
4.6 3 clusters=1 noise=30
4.6 4 clusters=1 noise=32
4.6 5 clusters=1 noise=32
4.6 6 clusters=1 noise=32
4.6 7 clusters=1 noise=32
4.6 8 clusters=1 noise=32
4.6 9 cluste

KeyboardInterrupt: 